In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
import random
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "drive/My Drive/Colab Notebooks/data/Datasets_Healthy_Older_People/S1_Dataset"
files = os.listdir(path)
gender_list = []
for i in range(len(files)):
    gender_list.append(files[i][-1])
dataset = []
for i in range(len(files)):
    lines = []
    file_path = path + "/" + files[i]
    file = open(file_path, "r")
    while True:
        every_line = file.readline()
        if not every_line:
            break
        every_line = every_line.strip().split(",")
        every_line.insert(-1, (1. if gender_list[i] is "M" else 0.))
        lines = [float(x) for x in every_line]
        dataset.append(lines)
random.shuffle(dataset)
values = []
classes = []
for i in range(len(dataset)):
    values.append(dataset[i][0:-1])
    classes.append(int(dataset[i][-1])-1)
values = np.array(values)
classes = np.array(classes)

In [ ]:
train_split = 0.7

train_value_set = values[0:int(train_split*len(values))]
train_class_set = classes[0:int(train_split*len(classes))]
test_value_set = values[int(train_split*len(values)):]
test_class_set = classes[int(train_split*len(classes)):]

In [ ]:
def select(condition, prediction, classes):
  count = 0
  length = 0
  for pre, classe in zip(prediction, classes):
    if pre == classe and pre == 3:
      count += 1
      length += 1
    elif classe == 3:
      length += 1
  print(confusion_matrix(classes, prediction))
  print(classification_report(classes, prediction))
  return count, length

In [ ]:
model = tree.DecisionTreeClassifier(max_depth=100, min_samples_leaf=1, min_samples_split=2)
model.fit(train_value_set, train_class_set)
prediction = np.array(model.predict(test_value_set))
count, length = select(3, prediction, test_class_set)
print(count / length)
print(count)
print(length)
print(np.sum(prediction == test_class_set)/len(test_class_set))
#在整个数据集进行测试
prediction = np.array(model.predict(values))
print(np.sum(prediction == classes)/len(classes))

0.8519061583577713
581
682
0.9854918141887395
0.995647428388883


In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/data/Models/DT/DTModel.pickle", "wb") as f:
    pickle.dump(model, f)

In [ ]:
model = pickle.load(open("/content/drive/My Drive/Colab Notebooks/data/Models/DT/DTModel.pickle", "rb"))
prediction = model.predict(test_value_set)
count, length = select(0, prediction, test_class_set)

[[ 4900    12     3    20]
 [   16  1458     0     5]
 [    6     0 15431     2]
 [   23     7     0   656]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4935
           1       0.99      0.99      0.99      1479
           2       1.00      1.00      1.00     15439
           3       0.96      0.96      0.96       686

    accuracy                           1.00     22539
   macro avg       0.98      0.98      0.98     22539
weighted avg       1.00      1.00      1.00     22539

